In [ ]:
!pip install transformers
!pip install evaluate==0.4.0
!pip install datasets==2.10.0
!pip install sentencepiece==0.1.97

from transformers import logging
logging.set_verbosity(logging.WARNING)

1. Classify car reviews

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Loading the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Creating the text classification pipeline with truncation
text_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, truncation=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# prompt: go to /content/drive/My Drive/Advanced_NLP/workspace/car_dataset
# read all the csv files, concatenate them in a way that the rows will be added to eaach other make one dataframe of the whole thing , name it df, count how many csv files you couldnt read

import pandas as pd
import os
import glob

# Define the directory containing the CSV files
directory = "/content/drive/My Drive/Advanced_NLP/workspace/car_dataset"

# Use glob to find all CSV files in the directory
csv_files = glob.glob(os.path.join(directory, "*.csv"))

# Initialize an empty list to store DataFrames
dfs = []

# Initialize a counter for unreadable files
unreadable_files = 0

# Iterate through the CSV files
for file in csv_files:
    try:
        # Read each CSV file into a DataFrame
        df_temp = pd.read_csv(file)
        # Append the DataFrame to the list
        dfs.append(df_temp)
    except Exception as e:
        print(f"Could not read file {file}: {e}")
        unreadable_files += 1

# Concatenate all DataFrames in the list into a single DataFrame
if dfs:
    df = pd.concat(dfs, ignore_index=True)
else:
    print("No CSV files were successfully read.")
    df = pd.DataFrame() # Create an empty DataFrame if none were read

print(f"Number of unreadable CSV files: {unreadable_files}")

# You can now work with the combined DataFrame 'df'
# For example, to display the first few rows:
print(df.head())

Could not read file /content/drive/My Drive/Advanced_NLP/workspace/car_dataset/Scraped_Car_Review_dodge.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

Could not read file /content/drive/My Drive/Advanced_NLP/workspace/car_dataset/Scrapped_Car_Review_Buick.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

Could not read file /content/drive/My Drive/Advanced_NLP/workspace/car_dataset/Scraped_Car_Review_jaguar.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

Could not read file /content/drive/My Drive/Advanced_NLP/workspace/car_dataset/Scrapped_Car_Reviews_Toyota.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

Could not read file /content/drive/My Drive/Advanced_NLP/workspace/car_dataset/Scraped_Car_Review_nissan.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

Coul

In [ ]:
df.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,on 07/24/18 09:42 AM (PDT),Joe,2017 FIAT 124 Spider Convertible Abarth 2dr Co...,Fantastic Car-Unsolvable Safety Issue,"The Abarth handles like a dream, shifts smart...",5.0
1,1,on 06/08/18 14:19 PM (PDT),John Hunter,2017 FIAT 124 Spider Convertible Lusso 2dr Con...,Honeymoon never ends,I have had this for a year now and still love...,5.0
2,2,on 05/31/18 06:08 AM (PDT),BJ,2017 FIAT 124 Spider Convertible Lusso 2dr Con...,"love this car, kid again!",havin a blast! gotta remind myself to be care...,5.0
3,3,on 04/03/18 09:56 AM (PDT),Michael Lang,2017 FIAT 124 Spider Convertible Abarth 2dr Co...,8 Months research and finally bought the ABARTH,I had concerns over buying a FIAT. Lets face ...,5.0
4,4,on 03/21/18 21:35 PM (PDT),steveromo,2017 FIAT 124 Spider Convertible Lusso 2dr Con...,"I now love spiders, 124 Spiders!",Know that interior storage space is minimal. ...,5.0


In [ ]:
print(len(df))

1978


In [ ]:
# prompt: in dataframe df, in column "Rating" turn the values to float, then ,if value is larger than 4, turn it into "POSITIVE", otherwise, turn it into "NEGATIVE", rename "Rating" column to "Class", then drop each column except "Review" and "Class"

df["Rating"] = pd.to_numeric(df["Rating"], errors='coerce')
df["Rating"] = df["Rating"].astype(float)
df["Class"] = ["POSITIVE" if i > 4 else "NEGATIVE" for i in df["Rating"]]
df = df[["Review", "Class"]]
print(df.head())

                                              Review     Class
0   The Abarth handles like a dream, shifts smart...  POSITIVE
1   I have had this for a year now and still love...  POSITIVE
2   havin a blast! gotta remind myself to be care...  POSITIVE
3   I had concerns over buying a FIAT. Lets face ...  POSITIVE
4   Know that interior storage space is minimal. ...  POSITIVE


In [ ]:
# prompt: in the column "Review" of df, they are strings, i want you to check to see if there is any of those have more than 512 words, cut the last words so the length is max 512 words, if the review is a float, remove that row from data frame and continue, just at the end count how many rows were removed because the review was a float

removed_rows = 0
for index, row in df.iterrows():
    review = row['Review']
    if isinstance(review, float):
        df.drop(index, inplace=True)
        removed_rows += 1
        continue
    words = review.split()
    if len(words) > 512:
        df.loc[index, 'Review'] = " ".join(words[:512])

print(f"Number of rows removed due to float reviews: {removed_rows}")

Number of rows removed due to float reviews: 123


<ipython-input-10-3b8bc33717ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index, inplace=True)
<ipython-input-10-3b8bc33717ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index, inplace=True)
<ipython-input-10-3b8bc33717ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index, inplace=True)
<ipython-input-10-3b8bc33717ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [ ]:
import pandas as pd

#file_path = "/content/drive/My Drive/Advanced_NLP/workspace/data/car_reviews.csv"
#df = pd.read_csv(file_path, delimiter=";")
#print(df.head())
real_labels = df["Class"].tolist()
references = [1 if label == "POSITIVE" else 0 for label in real_labels]
text = df["Review"].tolist()
sentiment = text_classifier(text)
predicted_labels = [text['label'] for text in sentiment]
predictions = [1 if label == "POSITIVE" else 0 for label in predicted_labels]
from sklearn.metrics import accuracy_score, f1_score
accuracy_result = accuracy_score(references, predictions)
f1_result = f1_score(references, predictions)

In [ ]:
accuracy_result

0.8005390835579514

In [ ]:
f1_result

0.8453177257525084

2. Translate a car review

In [ ]:
llm = pipeline("translation_en_to_es", model= "Helsinki-NLP/opus-mt-en-es")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
first_car_review = text[0]

In [ ]:
outputs = llm(first_car_review, clean_up_tokenization_spaces=True, max_length=30)

Your input_length: 134 is bigger than 0.9 * max_length: 30. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [ ]:
translated_review = outputs[0]['translation_text']

In [ ]:
print(translated_review)

El Abarth maneja como un sueño, cambia inteligente y fácil, se ve como grande, y sigue conduciendo simple. Obtienes miradas


In [ ]:
import evaluate
bleu = evaluate.load("bleu")

In [ ]:
# prompt: # prompt: using facebook/m2m100_418M or Facebook/m2m100_1.2B , in hugging face, generate two reference translations for the first car review stored in first_car_review. also remember the task is English to Spanish translation. use both Facebook models to generate different translations. make a data frame with both translations, a data frame that only has one column (with no name) and the entries of that column and the translations. name the data frame ref_df

from transformers import pipeline

# Load the two M2M100 models
translator_418M = pipeline("translation_en_to_es", model="facebook/m2m100_418M")
translator_1_2B = pipeline("translation_en_to_es", model="facebook/m2m100_1.2B")


# Generate translations using both models
translation_418M = translator_418M(first_car_review, clean_up_tokenization_spaces=True, max_length=30)[0]['translation_text']
translation_1_2B = translator_1_2B(first_car_review, clean_up_tokenization_spaces=True, max_length=30)[0]['translation_text']

# Create the DataFrame
ref_df = pd.DataFrame({
    "": [translation_418M, translation_1_2B]
})
ref_df

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Your input_length: 152 is bigger than 0.9 * max_length: 30. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Your input_length: 152 is bigger than 0.9 * max_length: 30. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


,
0,"El Abarth actúa como un sueño, se mueve inteli..."
1,"El Abarth maneja como un sueño, cambia de form..."


In [ ]:
# prompt: give me the first column on ref_df data frame
references = ref_df[""].tolist()

In [ ]:
bleu_score = bleu.compute(predictions=[translated_review], references= [references])

In [ ]:
print(bleu_score['bleu'])

0.4934494673001857


3. Ask a question about a car review

In [ ]:
from transformers import AutoTokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
question = "How long ago was the product purchased?"
context = text[2]
inputs = tokenizer(question, context, return_tensors= "pt")

tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

with torch.no_grad():
    outputs = model(**inputs)

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
answer

'2 months'

In [ ]:
print(text[2])

 havin a blast! gotta remind myself to be careful because it takes off like a rocket and handles like its riding on rails. just purchased 2 months ago , so don't have any comments regarding durability or reliability but so far its great. would recommend for anyone havin a midlife crisis , lol.


4. Summarize and analyze a car review

In [ ]:
from transformers import pipeline
llm = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import pandas as pd


real_labels = df["Class"].tolist()
references = [1 if label == "POSITIVE" else 0 for label in real_labels]
text = df["Review"].tolist()

In [ ]:
long_text = text[-1]
outputs = llm(long_text, max_length = 55, min_length = 50 , clean_up_tokenization_spaces = True)
summarized_text = outputs[0]['summary_text']

In [ ]:
long_text

" I have had several high end cars, my main focus is 4 door luxury sedan, combined with performance, this is my 2nd flying spur, 1st one was a 2006 model, 25k miles, this car gave me a lot of problems, mainly confined to the info-tainement system, though a few major engine problems did occur.Luckily it was under warranty, good job because it cost 20k plus had the front struts replaced, i sold it within 2 years.I purchased a high end Mercedes AMG SUV, what a mistake, how i missed the bentleys serene ride,composed road manners, the overall feeling of luxury, and secure ride. I had to buy another, there are no words to describe how it feels, other cars just don't compare.The Bentley is a very simple car  , there is no lane departure, brake assist,forward cameras, in fact you feel short changed in that department, but it you will not miss them, the is a simple elegance the other cars don't have, or replicate.So i purchased the best i could, 2012 Speed, what an improvement on the 2006, much

In [ ]:
summarized_text

"The Bentley is a very simple car. There is no lane departure, brake assist,forward cameras, in fact you feel short changed in that department, but it you will not miss them. The is a simple elegance the other cars don't have, or replicate"